<a href="https://colab.research.google.com/github/nomura-st/gas_questionnaire/blob/main/%E6%98%A0%E7%94%BB%E6%83%85%E5%A0%B1%E6%A4%9C%E7%B4%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [71]:
#############################
### カスタム情報
#############################
# 読み書き対象スプレッドシートID
ssID='xxxxxxxxx'
# 読み書き対象スプレッドシート シート名
sheetName='データ'
# データ表が始まる行の上の文字
dataStart='映画タイトル'

# 検索対象サイト
baseURL = 'https://eiga.com'

In [72]:
import requests
import re

from bs4 import BeautifulSoup  # BeautifulSoupクラスをインポート

# スクレイピング関数
def select(url, selector):
  html = requests.get(url)
  # 取得したHTMLをBeautifulSoupを使ってパースします。
  soup = BeautifulSoup(html.content, "html.parser")
  # パースしたHTMLから特定の要素を抽出します。結果はリストで返ってきます。
  elem = soup.select(selector)
  return elem

In [73]:
import urllib.parse

# 映画名で検索
def searchMovie(movieName):
  result = select(f'{baseURL}/search/{urllib.parse.quote(movieName)}/', "#rslt-movie ul a")
  # ID（を含むリンク）を取得
  movieIds = list(map(lambda html: [html.select("p[class='title']")[0].get_text(), html.get('href')], result))
  if len(movieIds) == 0:
    raise Exception(f'[{movieName}]という映画が見つかりませんでした。タイトルを修正してください')

  if len(movieIds) > 1:
    print(f'★★★注意★★★　[{movieName}]という映画で複数ヒットしました。最初にヒットした情報を設定します（候補：{", ".join(map(lambda l: l[0], movieIds))}）★★★')

  movieHtml = select(f'{baseURL}{movieIds[0][1]}', "main")[0]
  images = movieHtml.select(".hero-img img")
  infos = movieHtml.select(".movie-details .data")
  directors = movieHtml.select(".movie-staff [itemprop='director']")
  casts = movieHtml.select(".movie-cast .person [itemprop='name']")

  return {
      'name':movieIds[0][0],
      'image':images[0]["src"] if len(images) > 0 else "",
      'info':infos[0].get_text() if len(infos) > 0 else "",
      'director':directors[0].get_text() if len(directors) > 0 else "",
      'casts':list(map(lambda html:html.get_text(), casts))[:2]
  }


In [74]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
ss = gc.open_by_key(ssID)
sheet = ss.worksheet(sheetName)

# A列をループしてデータを特定
row = 0
dataFound = False
for aValue in sheet.col_values(1):
  row = row + 1
  if not dataFound:
    if aValue == dataStart:
      # データ表が見つかった
      dataFound = True
    continue
  
  if aValue == "":
    # データ表の開始から空欄が出た時点で終了
    break

  if sheet.cell(row, 2).value != "":
    # 情報が設定済みの映画はスキップ
    print(f'[情報] "{aValue}"はB列が空ではないのでスキップ')
    continue

  print(f'[情報] "{aValue}"で検索開始')
  # 検索
  movieInfo = searchMovie(aValue)

  print(f'  →"{movieInfo["name"]}"で更新開始')
  info = ""
  if movieInfo["director"].strip() != "":
    info = info + f'監督[{movieInfo["director"]}]  '
  if len(movieInfo["casts"]) > 0:
    info = info + f'演者[{",".join(movieInfo["casts"])}]  '
  if movieInfo["info"].strip() != "":
    info = info + f'{movieInfo["info"]}  '

  # 書き込み
  sheet.update_cell(row, 2, info)
  sheet.update_cell(row, 3, movieInfo["image"])
